In [ ]:
# -*- coding: utf-8 -*-
import xmltodict
import json
import os
import sqlite3

In [ ]:
con = sqlite3.connect("fns.db", check_same_thread=False)
cursor = con.cursor()
path = 'путь до разархивированной папки с xml-документами'

for file in os.listdir(path):
    with open(f'{path}/{file}', 'r') as f:
        a = f.read()
        xml = xmltodict.parse(a)
        xml_dict = json.loads(json.dumps(xml))['Файл']['Документ']
        try:
            for i in xml_dict:
                inn = i['СведНП']['@ИННЮЛ'].replace("\'", "'")
                org_name = i['СведНП']['@НаимОрг']
                date = i['@ДатаСост']
                try:
                    for j in i['СвУплСумНал']:
                        tax_name = j['@НаимНалог']
                        tax_amount = j['@СумУплНал']
                        #print(inn, org_name, tax_name, tax_amount)
                        cursor.execute(f"""INSERT INTO taxes (inn, org_name, tax_name, tax_amount, datetime) 
                                           VALUES (?,?,?,?,?)""", (inn, org_name, tax_name, tax_amount, date))
                        con.commit()
                except: 
                    tax_name = i['СвУплСумНал']['@НаимНалог']
                    tax_amount = i['СвУплСумНал']['@СумУплНал']
                    cursor.execute(f"""INSERT INTO taxes (inn, org_name, tax_name, tax_amount, datetime) 
                                       VALUES (?,?,?,?,?)""", (inn, org_name, tax_name, tax_amount, date))
                    con.commit()
        except:
            try:
                inn = xml_dict['СведНП']['@ИННЮЛ'].replace("\'", "'")
                org_name = xml_dict['СведНП']['@НаимОрг']
                date = xml_dict['@ДатаСост']
                try:
                    for j in xml_dict['СвУплСумНал']:
                        tax_name = j['@НаимНалог']
                        tax_amount = j['@СумУплНал']
                        cursor.execute(f"""INSERT INTO taxes (inn, org_name, tax_name, tax_amount, datetime) 
                                           VALUES (?,?,?,?,?)""", (inn, org_name, tax_name, tax_amount, date))
                        con.commit()
                except: 
                    tax_name = xml_dict['СвУплСумНал']['@НаимНалог']
                    tax_amount = xml_dict['СвУплСумНал']['@СумУплНал']
                    cursor.execute(f"""INSERT INTO taxes (inn, org_name, tax_name, tax_amount, datetime) 
                                       VALUES (?,?,?,?,?)""", (inn, org_name, tax_name, tax_amount, date))
                    con.commit()
            except:
                print(file)
                print(inn)
                print(xml_dict[i])